<a href="https://colab.research.google.com/github/Renan-Domingues/PyTorchRecipes/blob/main/SavingAndLoadingMultipleModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Saving and loading multiple models in one file using PyTorch

Saving and loading multiple models can be helpful for reusing models that was previously trained.

# Introduction
When saving a model comprised of multiple torch.nn.Module, it's important to save a dictionary of each model's state_dict and corresponding oprimizer.

 To load the models, first initialize the models and optimizers, then load the dictionary locally using torch.load().

 From here, you can easily access the saved items by simply querying the dictionary as you would expect. In this recipe, we will demonstrate how to save multiple models to one file using PyTorch.

In [1]:
# if wasn't available, I should instal torch

!pip install torch  # "!" is for instalation in Google Colab

# Steps

1. Import all necessary libraries for loading our data
2. Define and initialize the neural network
3. Initialize the optimizer
4. Save multiple models
5. Load multiple models


### 1. Import the libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

### 2. Define and initialize the neural network
This is a neural network for training images that I'm using in the examples for the basic recipes

In [3]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

netA = Net()
netB = Net()

### 3. Initialize the optimizer
We'll use a SGD with momentum

In [6]:
optimizerA = optim.SGD(netA.parameters(), lr=0.001, momentum=0.9)
optimizerB = optim.SGD(netB.parameters(), lr=0.001, momentum=0.9)

### 4. Save multiple models
Collect all relevant information and build your dictionary

In [7]:
# Specify a path to save to

PATH = "model.pt"

torch.save({
    'modelA_state_dict': netA.state_dict(),
    'modelB_state_dict': netB.state_dict(),
    'optimizerA_state_dict': optimizerA.state_dict(),
    'optimizerB_state_dict': optimizerB.state_dict(),
}, PATH)

### 5. Load Multiple models

In [8]:
# Remember to first initialize the models and optimizers, then load the dictionary locally.

modelA = Net()
modelB = Net()
optimModelA = optim.SGD(modelA.parameters(), lr=0.001, momentum=0.9)
optimModelB = optim.SGD(modelB.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
modelA.load_state_dict(checkpoint['modelA_state_dict'])
modelB.load_state_dict(checkpoint['modelB_state_dict'])
optimizerA.load_state_dict(checkpoint['optimizerA_state_dict'])
optimizerB.load_state_dict(checkpoint['optimizerB_state_dict'])

modelA.eval()
modelB.eval()

# or modelA.train()
# modelB.train()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

Remember to call model.eval() before runing the inference

or model.train() if we want to trein the model